<p style="font-weight:bold;"> <span style="font-size: 36px"> Queries </span> </p

In [0]:
#!import "./Extensions"

In [0]:
using System.Linq.Expressions;

# Exchange Rates

In [0]:
public static async Task<Dictionary<string, Dictionary<FxPeriod, double>>> GetExchangeRatesDictionaryAsync(this IQuerySource querySource, int year, int month)
    => (await querySource.Query<ExchangeRate>()
    .Where(x => x.Year == year - 1 && x.Month == MonthInAYear && x.FxType == FxType.Spot ||
                x.Year == year && x.Month == month)
    .ToArrayAsync())
    .ToDictionaryGrouped(x => x.Currency,
                         x => x.ToDictionary(y => (y.Year, y.Month, y.FxType) switch
                                             {
                                                 (_, _, _) when y.Year == year - 1 && y.Month == MonthInAYear && y.FxType == FxType.Spot    => FxPeriod.BeginningOfPeriod,
                                                 (_, _, _) when y.Year == year     && y.Month == month        && y.FxType == FxType.Average => FxPeriod.Average,
                                                 (_, _, _) when y.Year == year     && y.Month == month        && y.FxType == FxType.Spot    => FxPeriod.EndOfPeriod
                                             },
                                             y => y.FxToGroupCurrency));

# Current and Previous Parameters

In [0]:
public static async Task<T[]> LoadParameterAsync<T>(
    this IQuerySource querySource,
    int year,
    int month,
    Expression<Func<T, bool>> filterExpression = null ) 
    where T : IWithYearAndMonth
{
    return await querySource.Query<T>()
                     .Where(x => x.Year == year && x.Month <= month || x.Year < year)
                     .Where(filterExpression?? (Expression<Func<T, bool>>)(x => true))
                     .ToArrayAsync();
}

In [0]:
public static async Task<Dictionary<string, T>> LoadCurrentParameterAsync<T> (
    this IQuerySource querySource,
    Args args,
    Func<T, string> identityExpression,
    Expression<Func<T, bool>> filterExpression = null ) 
    where T : IWithYearMonthAndScenario
{
    return (await querySource.LoadParameterAsync<T>(args.Year, args.Month, filterExpression))
                             .Where(x => x.Scenario == args.Scenario || x.Scenario == null)
                             .GroupBy(x => identityExpression)
                             .Select(x => x.OrderByDescending(y => y.Year)
                                           .ThenByDescending(y => y.Month)
                                           .ThenByDescending(y => y.Scenario)
                                           .FirstOrDefault())
                             .ToDictionary(identityExpression);
}

In [0]:
public static async Task<Dictionary<string, Dictionary<int, T>>> LoadCurrentAndPreviousParameterAsync<T> (
    this IQuerySource querySource,
    Args args,
    Func<T, string> identityExpression,
    Expression<Func<T, bool>> filterExpression = null ) 
    where T : IWithYearMonthAndScenario
{
    var parameters = (await querySource.LoadParameterAsync<T>(args.Year, args.Month, filterExpression))
                    .Where(yc => yc.Scenario == args.Scenario || yc.Scenario == null)
                    .GroupBy(identityExpression);
                                         
    var ret = new Dictionary<string, Dictionary<int, T>>();
    foreach(var p in parameters)
    {
        var inner = ret.GetOrAdd(p.Key, _ => new Dictionary<int, T>());

        var currentCandidate = p.Where(x => x.Year == args.Year).OrderByDescending(x => x.Month).ThenByDescending(x => x.Scenario).FirstOrDefault();
        var previousCandidate = p.Where(x => x.Year < args.Year && x.Scenario == null).OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).FirstOrDefault();
        var currentCandidateBE = p.Where(x => x.Year <= args.Year && x.Scenario == null).OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).FirstOrDefault();
    
        inner.Add(CurrentPeriod, currentCandidate != null ? currentCandidate : previousCandidate);
        inner.Add(PreviousPeriod, previousCandidate != null ? previousCandidate : (currentCandidateBE != null ? currentCandidateBE : currentCandidate));
        // TODO: log error if currentCandidate is null
    }
    return ret;
}

<a id='yield-curve'></a>
# Yield Curve

## Locked-in

In [0]:
public static async Task<Dictionary<string, YieldCurve>> LoadLockedInYieldCurveAsync(this IQuerySource querySource, Args args, IEnumerable<DataNodeData> dataNodes)

{
    var lockedInYieldCurveByGoc = new Dictionary<string, YieldCurve>();
    foreach (var dn in dataNodes.Where(x => x.ValuationApproach == ValuationApproaches.BBA))
    {
        var argsNew = args with {Year = dn.Year, Month = dn.Month, Scenario = args.Scenario};
        var loadedYc = (await querySource.LoadCurrentParameterAsync<YieldCurve>(argsNew, x => x.Currency, x => x.Currency == dn.ContractualCurrency && x.Name == dn.YieldCurveName));
                                    
        if (!loadedYc.TryGetValue(dn.ContractualCurrency, out var lockedYc))
            ApplicationMessage.Log(Error.YieldCurveNotFound, dn.ContractualCurrency, argsNew.Year.ToString(), argsNew.Month.ToString());
        
        lockedInYieldCurveByGoc[dn.DataNode] = lockedYc;
    }
    
    return lockedInYieldCurveByGoc;
}

## Current

In [0]:
public static async Task<Dictionary<string, Dictionary<int, YieldCurve>>> LoadCurrentYieldCurveAsync(this IQuerySource querySource, Args args,
                                                                                                                                IEnumerable<DataNodeData> dataNodes)
{    
    var currentYieldCurveByGoc = new Dictionary<string,  Dictionary<int, YieldCurve>>();
    
    foreach (var dn in dataNodes)
    {
        var loadedYc = await querySource.LoadCurrentAndPreviousParameterAsync<YieldCurve>(args, 
                                                                               x => x.Currency,
                                                                               x => x.Currency == dn.ContractualCurrency 
                                                                                    && (dn.ValuationApproach == ValuationApproaches.VFA 
                                                                                                                ? x.Name == dn.YieldCurveName 
                                                                                                                : true));
        if(!loadedYc.TryGetValue(dn.ContractualCurrency, out var currentYcDict))
            ApplicationMessage.Log(Error.YieldCurveNotFound, dn.ContractualCurrency, args.Year.ToString(), args.Month.ToString());
    
        currentYieldCurveByGoc[dn.DataNode] = currentYcDict;
    }
    
    return currentYieldCurveByGoc;
}

# Data Node State

In [0]:
public static async Task<Dictionary<string, DataNodeState>> LoadDataNodeStateAsync(this IQuerySource querySource, Args args)
{
    return (await querySource.LoadCurrentAndPreviousParameterAsync<DataNodeState>(args, x => x.DataNode))
                             .Where(x => x.Value[CurrentPeriod].State != State.Inactive)
                             .ToDictionary(x => x.Key, x => x.Value[CurrentPeriod]);
}

# Data Nodes

In [0]:
public static async Task<Dictionary<string, DataNodeData>> LoadDataNodesAsync(this IQuerySource querySource, Args args)
{
    var dataNodeStates = await querySource.LoadCurrentAndPreviousParameterAsync<DataNodeState>(args, x => x.DataNode);
    var activeDataNodes = dataNodeStates.Where(kvp => kvp.Value[CurrentPeriod].State != State.Inactive).Select(kvp => kvp.Key);
    
    return (await querySource.Query<GroupOfContract>().Where(dn => activeDataNodes.Contains(dn.SystemName)).ToArrayAsync())
                            .ToDictionary(dn => dn.SystemName, dn => {
                                                                      var dnCurrentState = dataNodeStates[dn.SystemName][CurrentPeriod];
                                                                      var dnPreviousState = dataNodeStates[dn.SystemName][PreviousPeriod];
                                                                      return new DataNodeData(){Year = dnPreviousState.Year, 
                                                                                                  Month = dnPreviousState.Month,
                                                                                                  State = dnCurrentState.State,
                                                                                                  PreviousState = dnPreviousState.State,
                                                                                                  DataNode = dn.SystemName,
                                                                                                  ContractualCurrency  = dn.ContractualCurrency,
                                                                                                  FunctionalCurrency  = dn.FunctionalCurrency,
                                                                                                  LineOfBusiness  = dn.LineOfBusiness,
                                                                                                  ValuationApproach  = dn.ValuationApproach,
                                                                                                  OciType  = dn.OciType,
                                                                                                  Portfolio  = dn.Portfolio,
                                                                                                  AnnualCohort  = dn.AnnualCohort,
                                                                                                  LiabilityType  = dn.LiabilityType,
                                                                                                  Profitability  = dn.Profitability,
                                                                                                  Partner  = dn.Partner,
                                                                                                  IsReinsurance  = dn.GetType() == typeof(GroupOfReinsuranceContract),
                                                                                                  YieldCurveName = dn.YieldCurveName
                                                                                                  };
                                                                     }
                                         );
}

# Data Node Parameters

## Single data Node

In [0]:
public static async Task<Dictionary<string, Dictionary<int, SingleDataNodeParameter>>> LoadSingleDataNodeParametersAsync(this IQuerySource querySource, Args args)
{
    return await querySource.LoadCurrentAndPreviousParameterAsync<SingleDataNodeParameter>(args, x => x.DataNode);
}

## Inter data Node

In [0]:
public static async Task<Dictionary<string, Dictionary<int, HashSet<InterDataNodeParameter>>>> LoadInterDataNodeParametersAsync(this IQuerySource querySource, Args args)
{
    var identityExpressions = new Func<InterDataNodeParameter, string>[]{x => x.DataNode, x => x.LinkedDataNode,};
    var parameterArray = (await querySource.LoadParameterAsync<InterDataNodeParameter>(args.Year, args.Month));
    var parameters = identityExpressions.SelectMany(ie => parameterArray.GroupBy(ie));
                                         
    return parameters.SelectMany(p => p
                                 .GroupBy(x => x.DataNode != p.Key ? x.DataNode : x.LinkedDataNode)
                                 .Select(gg =>
                                         {
                                             var currentCandidate = gg.Where(x => x.Year == args.Year).OrderByDescending(x => x.Month).FirstOrDefault();
                                             var previousCandidate = gg.Where(x => x.Year < args.Year).OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).FirstOrDefault();
                                             return (key: p.Key,
                                                     currentPeriod: currentCandidate != null ? currentCandidate : previousCandidate,
                                                     previousPeriod: previousCandidate != null ? previousCandidate : currentCandidate);
                                         })
                                )
        .ToDictionaryGrouped(x => x.key,
                             x => new Dictionary<int, HashSet<InterDataNodeParameter>>{ {CurrentPeriod, x.Select(y => y.currentPeriod).ToHashSet()},
                                                                                       {PreviousPeriod, x.Select(y => y.previousPeriod).ToHashSet()}});
}

# AoC Step Configuration

In [0]:
public static async Task<IEnumerable<AocConfiguration>> LoadAocStepConfigurationAsync(this IQuerySource querySource, int year, int month)
    => (await querySource.LoadParameterAsync<AocConfiguration>(year, month))
            .GroupBy(x => (x.AocType, x.Novelty), 
                    (k, v) => v.OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).First()); 

In [0]:
public static async Task<Dictionary<AocStep, AocConfiguration>> LoadAocStepConfigurationAsDictionaryAsync(this IQuerySource querySource, int year, int month) 
    => (await querySource.LoadAocStepConfigurationAsync(year, month))
            .ToDictionary(x => new AocStep(x.AocType, x.Novelty)); 

# Data Variables

In [0]:
public static async Task<T[]> LoadPartitionedDataAsync<T,P>(this IDataSource querySource, Guid partition, Guid? partitionBackup = null)
    where T : IPartitioned
    where P : IPartition

{
    await querySource.Partition.SetAsync<P>(partition);
    // Temporary workaround for physical database: where clause is necessary
    var data = await querySource.Query<T>().Where(x => x.Partition == partition).ToArrayAsync();
    if(partitionBackup != null) await querySource.Partition.SetAsync<P>(partitionBackup);
    return data;
}

In [0]:
public static async Task<T[]> QueryPartitionedDataAsync<T,P>(this IWorkspace workspace, IDataSource dataSource, Guid targetPartition, Guid defaultPartition, string format)
    where T : IPartitioned
    where P : IPartition
{
    var isRelaxed = targetPartition != defaultPartition &&
                    ((format != ImportFormats.Cashflow && typeof(T).Name == nameof(IfrsVariable)) ||
                    (format == ImportFormats.Cashflow && typeof(T).Name == nameof(RawVariable)));

    var variablesFromWorkspace = await workspace.LoadPartitionedDataAsync<T,P>(targetPartition);
    if(!isRelaxed || variablesFromWorkspace.Any()) return variablesFromWorkspace;

    // For scenario re-calculation
    var variablesFromDataSource = await dataSource.LoadPartitionedDataAsync<T,P>(targetPartition);
    if(variablesFromDataSource.Any()) return variablesFromDataSource;

    // For scenarios related to parameters solely
    return await dataSource.LoadPartitionedDataAsync<T,P>(defaultPartition, targetPartition);
}